In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import stanza
import pandas as pd
import numpy as np

import codecs
import numpy as np
import nltk
import pycrfsuite
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
import re
import random
from tqdm.notebook import tqdm, trange

from stanza.utils.conll import CoNLL
from tqdm.notebook import tqdm

In [3]:
stanza.download('en', package='genia')
tok = stanza.Pipeline('en', package='genia', processors='tokenize')
nlp = stanza.Pipeline('en', package='genia', processors='tokenize,pos,lemma,depparse', tokenize_pretokenized=True)

2021-04-19 18:41:58 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | genia   |
| pos       | genia   |
| lemma     | genia   |
| depparse  | genia   |
| pretrain  | genia   |

2021-04-19 18:41:58 INFO: File exists: /home/wkalim/stanza_resources/en/tokenize/genia.pt.
2021-04-19 18:41:58 INFO: File exists: /home/wkalim/stanza_resources/en/pos/genia.pt.
2021-04-19 18:41:58 INFO: File exists: /home/wkalim/stanza_resources/en/lemma/genia.pt.
2021-04-19 18:41:59 INFO: File exists: /home/wkalim/stanza_resources/en/depparse/genia.pt.
2021-04-19 18:41:59 INFO: File exists: /home/wkalim/stanza_resources/en/pretrain/genia.pt.
2021-04-19 18:41:59 INFO: Finished downloading models and saved to /home/wkalim/stanza_resources.
2021-04-19 18:41:59 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | genia   |

/home/wkalim/.local/lib/python3.8/site-packa

In [4]:
found = 0
not_found = 0

In [6]:
def getIOBforLine(tagged_list, regex):
    global found
    global not_found
    
    word_list = []
    tag_list = []

    for i in tagged_list:
        word_list.append(i[0])
        if (i[0] in ['method', 'approach', 'model', 'algorithm', 'analysis']):
            tag_list.append(i[0])
        else:
            tag_list.append(i[1])

    sentence = ' '.join(word_list)
    sentence_in_tags = ' '.join(tag_list)
    rel = re.findall(regex, sentence_in_tags) or [sentence_in_tags]
    if len(rel[0]) != 3:
        not_found+=1
        return None
    found+=1
    matched = rel[0][0][1:-1].split(" ")
    iob_list = [0]*len(tag_list)
    for i in range(0, len(tag_list)):
        if tag_list[i] == matched[0]:
            number_of_matched = 0
            for j in range(0, len(matched)):
                if (i+j < len(tag_list)):
                    if tag_list[i+j] == matched[j]:
                        number_of_matched+=1
            if number_of_matched == len(matched):
                for j in range(0, len(matched)):
                    iob_list[i+j] = j+1

    for index in range(0, len(iob_list)):
        if iob_list[index] == 0:
            iob_list[index] = 'O'
        elif iob_list[index] == 1:
            iob_list[index] = 'B-method'
        else:
            iob_list[index] = 'I-method'
    return iob_list

def writeIOB2File(inputList, filename):
    with open(filename, 'w+') as fout:
        for i in dt_list:
            for j in i:
                output = ""
                if '\n' in j[0]:
                    output=j[0].strip('\n') + '\t' + j[2] + '\n\n'
                else:
                    output=j[0] + '\t' + j[2] + '\n'
                fout.write(output)
        
def convert2or3linesFormatToWorkableFormat(docs, number_of_lines):
    new_list = []
    for i in (''.join(docs).split('\n\n')):
        inner = i.split('\n')
        if '' in inner:
            inner.remove('')
        new_list.append(inner)
    # new_list.remove('')

    counter = [0] * 10

    for i in new_list:
        counter[len(i) - 1]+=1
    # print(counter)

    old_format_list = []
    old_format_list_with_filenames = []

    for i in new_list:
        # change this 5 to 4 or 6 depending on how many number of lines
        # 2 -> 4
        # 3 -> 5
        # 4 -> 6
        # ...
        if (len(i) == number_of_lines + 2):
    #         print(i)
            old_format_list.append(i[1]+'\n')
            old_format_list.append(i[-1]+'\n')
            old_format_list.append('--\n')
    return old_format_list


In [7]:
one_line_filename = './dataset/mm_1sentences'
two_line_filename = './dataset/mm_2sentences'
three_line_filename = './dataset/mm_3sentences'

# one_line_filename = './test/input.txt'
# two_line_filename = './test/28122020/mm_2sentences'
# three_line_filename = './test/28122020/mm_3sentences'


lines = 1

if (lines == 1):
    with open(one_line_filename, 'r') as fpm:
        docs = fpm.readlines()
elif (lines == 2):
    with open(two_line_filename, 'r') as fpm:
        docs = convert2or3linesFormatToWorkableFormat(fpm.readlines(), 2)
elif (lines == 3):
    with open(three_line_filename, 'r') as fpm:
        docs = convert2or3linesFormatToWorkableFormat(fpm.readlines(), 3)
else:
    docs = []
    print("Nope")

In [8]:
docs = list()
with open(one_line_filename, 'r') as fpm:
    docs+=fpm.readlines()
with open(two_line_filename, 'r') as fpm:
    docs+=convert2or3linesFormatToWorkableFormat(fpm.readlines(), 2)
with open(three_line_filename, 'r') as fpm:
    docs+=convert2or3linesFormatToWorkableFormat(fpm.readlines(), 3)
print(len(docs))

33093


In [9]:
regex = r"(( NN| JJ| NNP| JJS| JJR)+( method | analysis | algorithm | approach | model ))"


In [10]:
docs = [re.sub(' +', ' ', re.sub(r'[<>]*', '', re.sub(r'\[[^)]*\]|\([^)]*\)', '', line))).strip('\n') for line in docs if ((line.startswith('--') or line.startswith('This method')) == False)]
# docs = [re.sub(' +', ' ', re.sub(r'[<>]*', '', re.sub(r'\[[^)]*\]|\([^)]*\)', '', line))).strip('\n') for line in docs]
# docs = [line for line in docs if ((line.startswith('--') or line.startswith('This method')) == False)]


In [11]:
print(len(docs))

10974


In [12]:
for line in docs:
    print(line, end='\n\n')

Thereafter , the fluorescence of conjugated R.I.CK -Tat9 was measured using a Tecan fluorescence microplate reader with an excitation wavelength at 485 nm and an emission wavelength at 535 nm .

The resulting lymphocyte count per microliter is multiplied by the measured CD4 percentage to yield an absolute CD4 count per microliter of blood .

Frozen sections of various green fluorescent protein transgenic mouse heads were prepared using the film method developed by Kawamoto and Shimizu .

For example , the film method is useful for chasing donor cells that are labeled with GFP in an acceptor body .

Specifically , Kawamoto and Shimizu recommend 5- to 10-µm-thick sections for macroscopic examination and 2- to 5-µm-thick sections for microscopic examination .

Recently , Sado et al. introduced a novel way of producing monoclonal antibodies in rats called the “rat lymph node method” .

These considerations led us to explore an improved method of obtaining polyclonal antibodies in a much sh

Students were invited to choose one of these patients to present as a case study .

This study applied Q methodology , an established sorting method , to quantify subjective views on motivation .

Demographic data collected included gender , age group , employment contract , and the speciality of the clinician .

While this is not immediately under medical school control , universities can contribute meaningfully to discussions on balance of service and teaching commitments amongst health staff .

Therapists were invited to participate , and to encourage junior and senior colleagues , as well as friends to enroll .

The implementation of McGill University 's clerkship in Geriatric Medicine included a new method of evaluation to stimulate self-reflection in clerks as well as student-tutor interaction .

For the instances where mediation was demonstrated , we calculated the proportion of the relationship between relationship style and match in a primary care specialty that was mediated b

Another limitation of the study is that the list of diagnosis was based on chart review of all diagnoses already made by the primary care physician and no attempt was made to assess whether the diagnoses were correct .

One of these methods is a universalist model for the equivalence and adaptation of instruments that relate to health and quality of life .

Tracheoesophageal speech significantly improved the quality of life and limited the voice handicap imposed by total laryngectomy .

Higher scores reflected greater self-efficacy for strength .

Trained interviewers directly elicited utilities using the time trade-off method .

Among the qualitative methods , cognitive interviewing allows direct input from respondents on the item content , format , and understandability .

Construct validity of the IPFAM was obtained by correlating it with the physiotherapist 's evaluation of the severity of the child 's disability , and demonstrated good validity .

The utilization of the concept-ba

This prompted us to analyze the proposed direct interaction between Nef and cholesterol in vitro using NMR spectroscopy .

We chose to amplify virus on MAGI cells for 6 days prior to concentration on 20 % sucrose cushion .

Time-of-addition experiments using the HIV-1 fusion inhibitor T-20 and Env-pseudotyped luciferase reporter viruses were conducted to measure the entry kinetics of Env .

The genomic locations of the proviral integrations were determined using the splinkerette-based PCR method .

The length polymorphism analysis of 3 ' BLV flanking sequences was performed by making a run-off .

In fact , I designed a way to specifically mutate structured RNA signals and not the underlying DNA sequences , an idea that is largely based on a natural mechanism of regulated gene expression in bacteria that was worked out by Charles Yanofsky at Stanford University .

Microarray signal normalization was performed using the " median-normalization " procedure .

The length polymorphism analys

The patients ' physiologic condition prior to the BSI and on the day of BSI were assessed using the APACHE II score .

Besides clinical responsiveness , the effect of rFVIIa should be monitored with laboratory parameters , whereas appropriate tests are discussed controversially .

In the present approach , detection times of 4 hours for S. pneumoniae and N. meningitidis and 10 hours for Listeria monocytogenes using as little as 10 μl CSF were achieved .

Ordinarily , survival analysis assumes that events occur when they are observed .

Clinical and epidemiologic literature on hepatitis A were used to determine the durations of latent and infectious periods , vaccine efficacy , and duration of vaccine-derived immunity .

A similar procedure has been widely used for collecting spots of dried blood or saliva to screen for various infectious agents , and metabolic and genetic diseases , for the presence of specific genes , also for forensic purposes .

The serotypes , subtypes , LPS , clas

The proposed mechanism for the evaluation of unknowns within a partially sampled phylogeny is through the implementation of thresholds , chosen to separate intraspecific variation from interspecific differences .

Instead , infection with pilT affected the level of expression of a small subset of infection-responsive genes .

The authors first found sigma factor E sites the hard way—by comparing gene expression in two strains of E. coli that differed in their intrinsic level of sigma E-induced activity .

The distribution frequency for the four classes of spines was compared between the two groups using a chi-square test .

Based on the presence or absence of active wing molt , we determined subspecific identity and selected for the current analyses islandica individuals only .

Similar predictions are obtained for the ORs with two independent algorithms , TMHMM Server version 2.0 ) , and in analysis of the GR protein family .

Thus a technology that depends on the nonlinear absorption

These tools will be of great use to any researcher wishing to scan large intergenic regions for potential cis-regulatory modules .

SAX is based on the premise of transforming a time series into a corresponding sequence of symbols .

A target gene would be expressed in accordance with a Boolean function related to the selected genes .

From these rules , we obtained a data set of 126 genes with 100 % accuracy out of the 165 orthologous genes .

Inside each non-empty strip , the sequence associated with the smallest E-value is selected ) .

Target-template alignments reported by these methods were compared , evaluated , and ranked by the PCONS server to identify the preferred modeling templates and the consensus alignment .

 .

Conversely , there may have been some probe sets which are split by the presence of partial cDNAs in AceView that do not clearly define a poly-A site .

The dimensions of this mask are specific to the shape and size of that particular object .

Users are also ab

Conserved positions can be found for any subclade in the phylogeny , and the conservation and variability between any set of subclades can be analyzed .

A list of 251 tags were found that were expressed with the same fold difference as Tbx5 ; the genes corresponding to these tags may have a similar differential gene expression pattern in the limb or they may represent genes expressed in both forelimbs and hindlimbs .

It can be seen that as N increases , we also need a larger number of arrays to maintain the power of the statistical test when other parameters are fixed .

hart : Hartigan .

Four hundred EPKs could be associated with a Human Genome Organization Gene Nomenclature Committee name or description that implies previous knowledge about the potential function or subfamily membership of the sequence .

To make the verification of the classification results more straightforward we introduce a representation of classifiers in the form of simple rules .

First , a time-consuming a

Unfortunately , because of small numbers , we could not calculate rates adjusted by race or ethnicity , although disparities in cancer rates may exist for these groups and may warrant further investigation .

Despite the lack of evidence on maternal and fetal outcomes associated with attempted weight loss and actual weight loss during pregnancy , the behavior is not recommended and may indicate , like smoking and alcohol consumption , a high-risk pregnancy .

Study findings will provide a more accurate assessment of the relationship between weight and arthritis in the United States and will enhance related clinical medical interventions and public health initiatives .

To obtain the recombination fraction between two markers , these numbers were divided by two and multiplied by the number of nucleotides separating the two markers .

A non-parametric estimation technique applying the concept of ‘sample coverage’ was used to estimate the total number of OR genes in each of the nine avian

In [13]:
print(len(docs))

10974


In [14]:
sentence_tok_list = list()
for index in tqdm(range(0, len(docs))):
    sentence_nlp = nlp(tok(docs[index])).to_dict()[0]
    for i in range(0, len(sentence_nlp)):
        children = list()
        for j in range(0, len(sentence_nlp)):
            if (i == j):
                continue
            if sentence_nlp[j]["head"] == sentence_nlp[i]["id"]:
                children.append(sentence_nlp[j]["id"])
        sentence_nlp[i]["children"] = children
    sentence_tok_list.append(sentence_nlp)

  0%|          | 0/10974 [00:00<?, ?it/s]

In [15]:
import pickle

with open('sentences_tok.pkl', 'wb+') as f:
    pickle.dump(sentence_tok_list, f)

# ==============================================================================================================

In [36]:
with open('sentences_tok.pkl', 'rb') as f:
    a = pickle.load(f)
print(a[0])

[{'id': 1, 'text': 'Thereafter', 'lemma': 'thereafter', 'upos': 'ADV', 'xpos': 'RB', 'head': 10, 'deprel': 'advmod', 'misc': 'start_char=0|end_char=10', 'children': []}, {'id': 2, 'text': ',', 'lemma': ',', 'upos': 'PUNCT', 'xpos': ',', 'head': 10, 'deprel': 'punct', 'misc': 'start_char=11|end_char=12', 'children': []}, {'id': 3, 'text': 'the', 'lemma': 'the', 'upos': 'DET', 'xpos': 'DT', 'head': 4, 'deprel': 'det', 'misc': 'start_char=13|end_char=16', 'children': []}, {'id': 4, 'text': 'fluorescence', 'lemma': 'fluorescence', 'upos': 'NOUN', 'xpos': 'NN', 'head': 10, 'deprel': 'nsubj:pass', 'misc': 'start_char=17|end_char=29', 'children': [3, 8]}, {'id': 5, 'text': 'of', 'lemma': 'of', 'upos': 'ADP', 'xpos': 'IN', 'head': 8, 'deprel': 'case', 'misc': 'start_char=30|end_char=32', 'children': []}, {'id': 6, 'text': 'conjugated', 'lemma': 'conjugate', 'upos': 'VERB', 'xpos': 'VBN', 'head': 8, 'deprel': 'amod', 'misc': 'start_char=33|end_char=43', 'children': []}, {'id': 7, 'text': 'R.I.C

In [38]:
rules_count = dict()

def checkRule(tok_list, parent):
#     print(tok_list)
    children = parent['children']
    
    # Rule 1
    if set((rule_1 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound', 'conj']}).values()) == set(['compound', 'conj']):
        if ((len(rule_1.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_1.values()))):
            print(' '.join([tok_list[key]['text'] for key in tok_list.keys()]))
#             for key in tok_list.keys():
#                 print(tok_list[key])
            print("Rule 1")
            print(rule_1)
            word_list = []
            word_list.append(str(tok_list[list(rule_1.keys())[list(rule_1.values()).index('compound')]]['text']) + " " + str(parent['text']))
            for id in rule_1.keys():
                if (tok_list[id]['deprel'] in ['conj']):
                    word_list.append(tok_list[id]['text'] + " " + parent['text'])    
            print(word_list)
            print('\n')
            if 'rule_1' in rules_count:
                rules_count['rule_1'] = rules_count['rule_1'] + 1
            else:
                rules_count['rule_1'] = 0
    
    # Rule 2
#     rule_2 = {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound', 'amod']}
    elif set((rule_2 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['compound', 'amod']}).values()) == set(['compound', 'amod']):
        if ((len(rule_2.values()) > 0) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_2.values()))):
            print(' '.join([tok_list[key]['text'] for key in tok_list.keys()]))
#             for key in tok_list.keys():
#                 print(tok_list[key])
            print("Rule 2")
            print(rule_2)
            word_list = []
#             word_list.append(str(tok_list[list(rule_2.keys())[list(rule_2.values()).index('compound')]]['text']) + " " + str(parent['text']))
            word_string = ""
            for id in rule_2.keys():
#                 if (tok_list[id]['deprel'] in ['amod']):
#                     word_list.append(tok_list[id]['text'] + " " + parent['text'])    
                if (tok_list[id]['text'] in ['=']):
                    continue
                word_string+=tok_list[id]['text'] + " "
            word_string+=parent['text']
            word_list.append(word_string)
            print(word_list)
            print('\n')
            if 'rule_2' in rules_count:
                rules_count['rule_2'] = rules_count['rule_2'] + 1
            else:
                rules_count['rule_2'] = 0
    
    elif (set((rule_3 := {id: tok_list[id]['deprel'] for id in children if tok_list[id]['deprel'] in ['amod']}).values()) == set(['amod'])) and (parent['deprel'] == 'conj'):
        if ((len(rule_3.values()) > 1) and (all(x in [tok_list[id]['deprel'] for id in children] for x in rule_3.values()))):
            print(' '.join([tok_list[key]['text'] for key in tok_list.keys()]))
#             for key in tok_list.keys():
#                 print(tok_list[key])
            print("Rule 3")
            print(rule_3)
            word_list = []
#             word_list.append(str(tok_list[list(rule_2.keys())[list(rule_2.values()).index('compound')]]['text']) + " " + str(parent['text']))
            word_string = ""
            for id in rule_3.keys():
#                 if (tok_list[id]['deprel'] in ['amod']):
#                     word_list.append(tok_list[id]['text'] + " " + parent['text'])    
                if (tok_list[id]['deprel']) in ['amod']:
                    if (tok_list[id]['text'] in ['=']):
                        continue
                    word_string+=tok_list[id]['text'] + " "
            word_string+=parent['text']
            
            for id in children:
                if tok_list[id]['deprel'] == 'acl':
                    new_tok = tok_list[id]
                    word_string+= " " + new_tok['text']
                    if len(new_tok['children']) > 0:
                        newer_tok = tok_list[new_tok['children'][0]]
                        for i in newer_tok['children']:
                            newest_tok = tok_list[i]
                            if newest_tok['deprel'] == 'case':
                                word_string+=" " + newest_tok['text'] + " " + newer_tok['text']
            
            word_list.append(word_string)
            print(word_list)
            print('\n')
            if 'rule_3' in rules_count:
                rules_count['rule_3'] = rules_count['rule_3'] + 1
            else:
                rules_count['rule_3'] = 0

# {'id': 29, 'text': '3', 'lemma': '3', 'upos': 'X', 'xpos': 'LS', 'head': 33, 'deprel': 'dep', 'misc': 'start_char=161|end_char=162', 'children': [30]}
# {'id': 30, 'text': ')', 'lemma': ')', 'upos': 'PUNCT', 'xpos': '-RRB-', 'head': 29, 'deprel': 'punct', 'misc': 'start_char=163|end_char=164', 'children': []}
# {'id': 31, 'text': 'Robust', 'lemma': 'robust', 'upos': 'ADJ', 'xpos': 'JJ', 'head': 33, 'deprel': 'amod', 'misc': 'start_char=165|end_char=171', 'children': []}
# {'id': 32, 'text': 'Multi-Array', 'lemma': 'multi-array', 'upos': 'ADJ', 'xpos': 'JJ', 'head': 33, 'deprel': 'amod', 'misc': 'start_char=172|end_char=183', 'children': []}
# {'id': 33, 'text': 'Analysis', 'lemma': 'analysis', 'upos': 'NOUN', 'xpos': 'NN', 'head': 14, 'deprel': 'conj', 'misc': 'start_char=184|end_char=192', 'children': [29, 31, 32, 34]}
# {'id': 34, 'text': 'developed', 'lemma': 'develop', 'upos': 'VERB', 'xpos': 'VBN', 'head': 33, 'deprel': 'acl', 'misc': 'start_char=193|end_char=202', 'children': [36]}
# {'id': 35, 'text': 'by', 'lemma': 'by', 'upos': 'ADP', 'xpos': 'IN', 'head': 36, 'deprel': 'case', 'misc': 'start_char=203|end_char=205', 'children': []}
# {'id': 36, 'text': 'Irizarry', 'lemma': 'Irizarry', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 34, 'deprel': 'obl', 'misc': 'start_char=206|end_char=214', 'children': [35, 38]}
# {'id': 37, 'text': 'et', 'lemma': 'et', 'upos': 'X', 'xpos': 'FW', 'head': 38, 'deprel': 'compound', 'misc': 'start_char=215|end_char=217', 'children': []}
# {'id': 38, 'text': 'al.', 'lemma': 'al.', 'upos': 'X', 'xpos': 'FW', 'head': 36, 'deprel': 'advmod', 'misc': 'start_char=218|end_char=221', 'children': [37]}

            
def convertWord2WordDict(sentence):
    return {word['id']:word for word in sentence}

def getWordsFromDepTree(tok_list, parent = None):
    dependency_list = list(['acl'])
    initial_word_list = list(['method', 'analysis', 'algorithm', 'approach', 'model', 'preparation', 'strategy', 'technique', 'normalization', 'quantification'])
#     initial_word_list = list(['quantification']), 
     
    if parent == None:
        for tok in tok_list.values():
            if tok['deprel'] == 'root':
                parent = tok
                getWordsFromDepTree(tok_list, parent)
    else:
#         print(parent)
        
        for i in parent['children']:
            tok = tok_list[i]
            if tok['text'].lower() in initial_word_list:
                checkRule(tok_list, tok)
#                 getWordsFromDepTree(tok_list, tok)
#                 pass
            else:
                getWordsFromDepTree(tok_list, tok)
#         for i in parent['children']:
#             for tok in tok_list:
#                 if i == tok['id']:
#                     if (tok['text'] in initial_word_list):
#                         if (tok['deprel'] in dependency_list):
#                             getWordsFromDepTree(tok_list, tok)
#                     else:
#                         getWordsFromDepTree(tok_list, tok)
# print(sentence_tok_list[3])
# for sentence in sentence_tok_list:
#     getWordsFromDepTree(sentence)
# getWordsFromDepTree(convertWord2WordDict(sentence_tok_list[3]))
for sentence in sentence_tok_list:
#     sentence_dict = {word['id']:word for word in sentence}
#     print(sentence)
    getWordsFromDepTree(convertWord2WordDict(sentence))
print(rules_count)
print("Total number of sentences: " + str(len(sentence_tok_list)))    
#     print(sentence_dict)
#     for i in sentence_dict.values():
#         print(i)
# #     for word in sentence:
# #         print(word)
#     break

The Principal Component Analysis technique was used to summarize and discover patterns of inter-correlations among the studied measures .
Rule 2
{2: 'amod', 3: 'compound', 4: 'compound'}
['Principal Component Analysis technique']


In the present study , 10 RTK-derived ASV were screened side by side in the high-throughput CIA model , using replication-incompetent adenoviruses as a delivery and in vivo expression system .
Rule 2
{16: 'amod', 17: 'compound'}
['high-throughput CIA model']


To develop a reproducible and accurate method of gene expression analysis on synovial biopsies , we evaluated and validated real-time quantitative PCR on very small synovial tissue samples using a novel cell-based standard curve technique .
Rule 2
{31: 'amod', 32: 'amod', 33: 'amod', 34: 'compound'}
['novel cell-based standard curve technique']


Cycle-to-cycle fluorescence emission readings were monitored and quantified using the second derivative maximum method from the MX3000P Real Time software pac

['comparative threshold cycle method']


Additionally , a cut-off score of 6+ for the BSRS-5 was determined for psychiatric disorders using receiver operating characteristic analysis .
Rule 2
{17: 'compound', 18: 'compound', 19: 'amod'}
['receiver operating characteristic analysis']


The propensity score-adjusted bootstrapping method was also used to test the mean differences of costs for the 2 high level crisis event groups : patients with 2 or more crisis-event variables and patients with 3 or more crisis-event variables .
Rule 2
{2: 'compound', 3: 'amod', 4: 'compound'}
['propensity score-adjusted bootstrapping method']


We used a previously described match-merge method within SAS statistical software to merge the data sets on the basis of non-unique identifiers available in both datasets .
Rule 2
{5: 'amod', 6: 'compound'}
['described match-merge method']


The Holt-Winters forecasting method was used to predict the mortality rate per 100,000 person-years during and after the SAR

['analytical best-tag Bonferroni method']


The random walk method was used to select households .
Rule 2
{2: 'amod', 3: 'compound'}
['random walk method']


Because we selected the partners of the cases as control individuals , and because it turned out , as expected , that couples tend to travel together , we performed a conditional logistic regression analysis to calculate odds ratios for the relation between travel and venous thrombosis .
Rule 2
{32: 'amod', 33: 'amod', 34: 'compound'}
['conditional logistic regression analysis']


We employed cross-sectional time series analysis by generalized least-square regression for random effects using the xtreg command in STATA 8.0 software .
Rule 2
{3: 'amod', 4: 'compound', 5: 'compound'}
['cross-sectional time series analysis']


The data on sexual partnerships and condom use were collected using the Informal Confidential Voting Interview method for 75 % of respondents selected at random in the first round of the survey .
Rule 2
{13: 'am

Its detection requires a different analysis technique , allowing for averaging signal 's energy irrelevant of the phase2 .
Rule 2
{5: 'amod', 6: 'compound'}
['different analysis technique']


In an attempt to enhance the active contours method a cubic spline interpolation technique was used in to identify an initial contour based on four user-defined points .
Rule 2
{11: 'amod', 12: 'compound', 13: 'compound'}
['cubic spline interpolation technique']


This system used digital video analysis and position information from infrared beam breaking to assess sleep and wake behaviours in mice .
Rule 1
{5: 'compound', 9: 'conj'}
['video analysis', 'information analysis']


Incidence rate ratios and corresponding 95 percent confidence intervals for endometrial cancer were estimated in the age-adjusted and multivariate case-cohort analyses with categorized and continuous alcohol and cigarette smoking variables , using the Cox proportional hazards model .
Rule 2
{34: 'compound', 35: 'amod', 36: 

Here , we present DISCLOSE , an exploratory application that benchmarks clustering methods using functional annotations and a de novo motif discovery algorithm .
Rule 2
{19: 'amod', 21: 'compound', 22: 'compound'}
['de motif discovery algorithm']


A straight forward approach for the isolation of differentially expressed genes was achieved by the " Suppression Subtractive Hybridization " method .
Rule 2
{16: 'compound', 17: 'compound', 18: 'amod', 19: 'compound', 20: 'compound'}
['" Suppression Subtractive Hybridization " method']


The mismatch-specific endonuclease method for beta-thalassemia mutations detection is merely as a model for other disease genes for which large scale , high-throughput mutation scanning .
Rule 2
{2: 'amod', 3: 'compound'}
['mismatch-specific endonuclease method']


However , neither the original RM method provide enough data to calculate the duration of all cell cycle phases , consequently the duration of the whole cycle , and GF of a tumour .
Rule 2
{5: 'a

Synthesis of particles outside this range are not possible using the reverse micelle method .
Rule 2
{12: 'amod', 13: 'compound'}
['reverse micelle method']


We therefore set out to develop an RT-PCR based method to quantitate CD4 and CD8 mRNA in the hopes that this could be used to predict cell counts .
Rule 2
{8: 'compound', 9: 'amod'}
['RT-PCR based method']


The students were then introduced to their task , which was to apply the basic steps associated with the grounded theory approach in analysing the material provided to derive positive and negative attributes of operational research for influencing communicable disease policy and practice in southern Africa .
Rule 2
{20: 'amod', 21: 'compound'}
['grounded theory approach']


In a separate analysis , ITN use was analysed by individual , that is , by ' treatment-per-protocol ' logistic regression analysis , under the rationale that households typically had more than one bednet and because education uptake was highly individualis

In [37]:
def checkRule(tok_list, parent):
    children = parent['children']
    # Rule 1
#     rule_1 = {id: deprel for deprel in ['compound', 'conj']}
    rule_1 = {tok['id']: tok['deprel'] for deprel in ['compound', 'conj'] for id in children for tok in tok_list if (id == tok['id'] and tok['deprel'] == deprel)}
    print(rule_1)
    if set(rule_1.values()).issubset([tok['deprel'] for tok in tok_list for id in children if id == tok['id']]):
        print(True)
        ids = list(rule_1.keys())
        return [parent['text'] + " " +]
#         for id in children:
#             for tok in tok_list:
#                 if (id == tok['id'])
        
#         return []

def getWordsFromDepTree(tok_list, parent = None):
    dependency_list = list(['acl'])
    initial_word_list = list(['method', 'analysis', 'algorithm', 'approach', 'model', 'preparation'])
     
    if parent == None:
        for tok in tok_list:
            if tok['deprel'] == 'root':
                parent = tok
                getWordsFromDepTree(tok_list, parent)
    else:
#         print(parent)
        
        for i in parent['children']:
            for tok in tok_list:
                if (i == tok['id']):
                    if tok['text'] in initial_word_list:
                        checkRule(tok_list, tok)
                    else:
                        getWordsFromDepTree(tok_list, tok)
#         for i in parent['children']:
#             for tok in tok_list:
#                 if i == tok['id']:
#                     if (tok['text'] in initial_word_list):
#                         if (tok['deprel'] in dependency_list):
#                             getWordsFromDepTree(tok_list, tok)
#                     else:
#                         getWordsFromDepTree(tok_list, tok)
# print(sentence_tok_list[3])
# for sentence in sentence_tok_list:
#     getWordsFromDepTree(sentence)
getWordsFromDepTree(sentence_tok_list[3])

{8: 'compound', 11: 'conj'}
True
[8, 11]


In [12]:
# line_tok = nlp(tok(docs[0])).to_dict()[0]
# # print(line_tok)
# print(' '.join([word.get('text') for word in line_tok]))
# print(' '.join([word.get('lemma') for word in line_tok]))
# print(' '.join([word.get('upos') for word in line_tok]))
# print(' '.join([word.get('xpos') for word in line_tok]))
# print(' '.join([word.get('deprel') for word in line_tok]))

sentence_tok_list = list()

for index in range(0, len(docs)):
    line_tok = nlp(tok(docs[index])).to_dict()[0]
    sentence_info = dict()
    sentence_info['id'] = index+1
    sentence_info['text'] = ' '.join([word.get('text') for word in line_tok])
    sentence_info['lemma'] = ' '.join([word.get('lemma') for word in line_tok])
    sentence_info['upos'] = ' '.join([word.get('upos') for word in line_tok])
    sentence_info['xpos'] = ' '.join([word.get('xpos') for word in line_tok])
    sentence_info['deprel'] = ' '.join([word.get('deprel') for word in line_tok])
  
    sentence_tok_list.append(sentence_info)
    print(sentence_info)
print(sentence_tok_list)

{'id': 1, 'text': 'Aliquots were withdrawn at different time points and centrifuged at 14,000 × g for 90 min with a 10 kDa cut-off Microcon™ filter .', 'lemma': 'aliquot be withdraw at different time point and centrifuge at 14,000 × g for 90 min with a 10 kda cut-off microcon™ filter .', 'upos': 'NOUN AUX VERB ADP ADJ NOUN NOUN CONJ VERB ADP NUM NOUN NOUN ADP NUM NOUN ADP DET NUM NOUN NOUN NOUN NOUN PUNCT', 'xpos': 'NNS VBD VBN IN JJ NN NNS CC VBN IN CD NN NN IN CD NN IN DT CD NN NN NN NN .', 'deprel': 'nsubj:pass aux:pass root case amod compound obl cc conj case nummod compound obl case nummod obl case det compound amod compound compound nmod punct'}
{'id': 2, 'text': 'In order to calculate the absolute CD4 T-cell cell count , a complete blood cell count must be done to determine the absolute white blood cell count .', 'lemma': 'in order to calculate the absolute cd4 t-cell cell count , a complete blood cell count must be do to determine the absolute white blood cell count .', 'upos':

{'id': 15, 'text': 'Silage , hay , concentrate , blood and muscle samples were analysed for selenium at the National Veterinary Institute , Norway .', 'lemma': 'silage , hay , concentrate , blood and muscle sample be analyse for selenium at the national Veterinary Institute , norway .', 'upos': 'NOUN PUNCT NOUN PUNCT VERB PUNCT NOUN CONJ NOUN NOUN AUX VERB ADP NOUN ADP DET ADJ PROPN PROPN PUNCT NOUN PUNCT', 'xpos': 'NN , NN , VBP , NN CC NN NNS VBD VBN IN NN IN DT JJ NNP NNP , NN .', 'deprel': 'nsubj punct conj punct root punct obj cc compound nsubj:pass aux:pass conj case obl case det amod compound obl punct appos punct'}
{'id': 16, 'text': 'At present the genus Campylobacter contains 17 species , four of which have been further divided into eight subspecies http : / / www.bacterio.cict.fr / .', 'lemma': 'at present the genus campylobacter contain 17 species , four of which have be further divide into eight subspecy http : / / www.bacterio.cict.fr / .', 'upos': 'ADP ADJ DET NOUN NOUN 

{'id': 28, 'text': 'In addition to improving the method , comparisons between other molecular typing methods such as FAFLP or a sequence based typing scheme , would ultimately determine the validity of the MLVA assay as molecular epidemiology tool for L. interrogans .', 'lemma': 'in addition to improve the method , comparison between other molecular typing method such as faflp or a sequence base typing scheme , would ultimately determine the validity of the mlva assay as molecular epidemiology tool for l. interrogans .', 'upos': 'ADP NOUN ADP VERB DET NOUN PUNCT NOUN ADP ADJ ADJ NOUN NOUN ADJ ADP NOUN CONJ DET NOUN VERB NOUN NOUN PUNCT AUX ADV VERB DET NOUN ADP DET NOUN NOUN ADP ADJ NOUN NOUN ADP X X PUNCT', 'xpos': 'IN NN TO VBG DT NN , NNS IN JJ JJ NN NNS JJ IN NN CC DT NN VBN NN NN , MD RB VB DT NN IN DT NN NN IN JJ NN NN IN FW FW .', 'deprel': 'case obl mark advcl det obj punct nsubj case amod amod compound nmod case fixed nmod cc det conj case compound nmod punct aux advmod root d

{'id': 40, 'text': 'Most retrovirus vectors are derived from the Moloney murine leukemia virus , and were the first to be developed for human use .', 'lemma': 'most retrovirus vector be derive from the moloney murine leukemia virus , and be the first to be develop for human use .', 'upos': 'ADJ NOUN NOUN AUX VERB ADP DET NOUN ADJ NOUN NOUN PUNCT CONJ VERB DET ADJ PART AUX VERB ADP ADJ NOUN PUNCT', 'xpos': 'JJS NN NNS VBP VBN IN DT NN JJ NN NN , CC VBD DT JJ TO VB VBN IN JJ NN .', 'deprel': 'amod compound nsubj:pass aux:pass root case det compound amod compound obl punct cc cop det conj mark aux:pass acl case amod obl punct'}
{'id': 41, 'text': 'The study has several potential limitations .', 'lemma': 'the study have several potential limitation .', 'upos': 'DET NOUN VERB ADJ ADJ NOUN PUNCT', 'xpos': 'DT NN VBZ JJ JJ NNS .', 'deprel': 'det nsubj root amod amod obj punct'}
{'id': 42, 'text': 'Inclusion criteria were the availability of complete sets of MRIs with STIR and T1-post-Gd seque

{'id': 54, 'text': 'Joint fluid was obtained from 10 healthy control donors , 10 patients with OA , and 10 with RA .', 'lemma': 'joint fluid be obtain from 10 healthy control donor , 10 patient with oa , and 10 with ra .', 'upos': 'NOUN NOUN AUX VERB ADP NUM ADJ ADJ NOUN PUNCT NUM NOUN ADP NOUN PUNCT CONJ NUM ADP NOUN PUNCT', 'xpos': 'NN NN VBD VBN IN CD JJ JJ NNS , CD NNS IN NN , CC CD IN NN .', 'deprel': 'compound nsubj:pass aux:pass root case nummod amod amod obl punct nummod conj case nmod punct cc conj case nmod punct'}
{'id': 55, 'text': 'Some infiltrating cells as well as the cartilage surface stained positively for citrulline and the major sources of citrullinated proteins in the arthritic joint were extracellular deposits , presumably fibrin deposits .', 'lemma': 'some infiltrate cell as well as the cartilage surface stain positively for citrulline and the major source of citrullinated protein in the arthritic joint be extracellular deposit , presumably fibrin deposit .', 'upo

...

{'id': 258, 'text': 'The stoichiometric distribution towards a targeted product using a specific elementary mode provides insights into the route that needs engineering to enhance productivity .', 'lemma': 'the stoichiometric distribution towards a target product use a specific elementary mode provide insight into the route that need engineer to enhance productivity .', 'upos': 'DET ADJ NOUN ADP DET VERB NOUN VERB DET ADJ ADJ NOUN VERB NOUN ADP DET NOUN PRON VERB VERB PART VERB NOUN PUNCT', 'xpos': 'DT JJ NN IN DT VBN NN VBG DT JJ JJ NN VBZ NNS IN DT NN WDT VBZ VBG TO VB NN .', 'deprel': 'det amod nsubj case det amod nmod acl det amod amod obj root obj case det nmod nsubj acl:relcl xcomp mark xcomp obj punct'}
{'id': 259, 'text': 'Struyf et al. followed the same idea but advocated a different distance measure , which is easier to interpret and is guaranteed to be positive .', 'lemma': 'struyf et al. follow the same idea but advocate a different distance measure , which be easier to int

{'id': 272, 'text': 'Pellegrini et al. firstly proposed a phylogenetic profile described as a string of bits , each bit representing the absence or presence of an homologous gene in a given genome .', 'lemma': 'pellegrini et al. firstly propose a phylogenetic profile describe as a string of bit , each bit represent the absence or presence of a homologous gene in a given genome .', 'upos': 'NOUN X X ADV VERB DET ADJ NOUN VERB ADP DET NOUN ADP NOUN PUNCT DET NOUN VERB DET NOUN CONJ NOUN ADP DET ADJ NOUN ADP DET ADJ NOUN PUNCT', 'xpos': 'NN FW FW RB VBD DT JJ NN VBN IN DT NN IN NNS , DT NN VBG DT NN CC NN IN DT JJ NN IN DT JJ NN .', 'deprel': 'nsubj compound advmod advmod root det amod obj acl case det obl case nmod punct det appos acl det obj cc conj case det amod nmod case det amod nmod punct'}
{'id': 273, 'text': 'The activity of the compound was confirmed by FACS analysis where an accumulation of cells in G1 phase of cell cycle was observed , associated with a reduction in DNA duplica

{'id': 284, 'text': 'The docking energy was calculated for the sum of three energy , electrostatic energies , van der Waals energies and energy of the ligand .', 'lemma': 'the docking energy be calculate for the sum of three energy , electrostatic energy , van der waal energy and energy of the ligand .', 'upos': 'DET NOUN NOUN AUX VERB ADP DET NOUN ADP NUM NOUN PUNCT ADJ NOUN PUNCT NOUN NOUN NOUN NOUN CONJ NOUN ADP DET NOUN PUNCT', 'xpos': 'DT NN NN VBD VBN IN DT NN IN CD NN , JJ NNS , NN NN NN NNS CC NN IN DT NN .', 'deprel': 'det compound nsubj:pass aux:pass root case det obl case nummod nmod punct amod conj punct compound compound compound conj cc conj case det nmod punct'}
{'id': 285, 'text': 'By choosing a target region next to catalytic site in the docking study and then evolving the peptide ligand on the basis of binding indicators , we succeeded in obtaining a non-competitive peptide inhibitor .', 'lemma': 'by choose a target region next to catalytic site in the docking study a

{'id': 298, 'text': 'BSputnik { 1 , -6 , 7 } TRF { 2 , 7 , 7 ; 20 } MrepsSTARRepeatMaskerASputnik { 1 , -6 , 7 } -34.94 20.4 9.51 7.37 TRF { 2 , 7 , 7 ; 20 } 85.61 -45.3 17.26 12.6 Mreps82.63 80.85 -33.34 24.63 STAR95.29 93.92 93.61 -57.98 RepeatMasker100 97.89 97.64 82.13 -Proportion of the total number of detections of algorithm A also detected by algorithm B in the human X chromosome .', 'lemma': 'bsputnik { 1 , -6 , 7 } trf { 2 , 7 , 7 ; 20 } mrepsstarrepeatmaskerasputnik { 1 , -6 , 7 } -34.94 20.4 9.51 7.37 trf { 2 , 7 , 7 ; 20 } 85.61 -45.3 17.26 12.6 mreps82.63 80.85 -33.34 24.63 star95.29 93.92 93.61 -57.98 repeatmasker100 97.89 97.64 82.13 -proportion of the total number of detection of algorithm a also detect by algorithm b in the human x chromosome .', 'upos': 'NOUN NOUN NUM PUNCT NUM PUNCT NUM NOUN NOUN PUNCT NUM PUNCT NUM PUNCT NUM PUNCT NUM NOUN NOUN NOUN NUM PUNCT NUM PUNCT NUM NOUN NUM NUM NUM NUM NOUN PUNCT NUM PUNCT NUM PUNCT NUM PUNCT NUM NOUN NUM NUM NUM NUM NOUN NU

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(sentence_tok_list)

In [ ]:
file_name = 'stanza.data.conllu'
df.to_csv(file_name, sep='\t', index=False)

In [ ]:
df.head()

In [ ]:
tokenized_list = []
for line in docs:
    tokline = nlp(tok(line))
    print(tokline.to_dict()[0])
    # Add new row to df for each iteration of the loop
#     tokenized_list.append(tokline[0])
    break

In [ ]:
### Tokenize the data using Tokenize pipeline
tokdoc = (tok(docs))

### Generate new pre-tokenized data, one sentence per line
tokenizeddoc = ''
for sent in tqdm(tokdoc.sentences):
    for token in sent.tokens:
        tokenizeddoc = tokenizeddoc + token.text + ' '
    tokenizeddoc = tokenizeddoc + '\n'
print (tokenizeddoc)

### Modify the tokenized data (hyphens, biochem names, ...)
tokenized_doc = tokenizeddoc.replace(" - ", "-")

### Produce conll using stanza pipeline following Stanza website
doc = (nlp(tokenized_doc))
dicts = doc.to_dict()

In [ ]:
df = pd.DataFrame(dicts)

In [ ]:
df.head()

In [ ]:
conll = CoNLL.convert_dict(dicts)

### Output conll in conllu format used by PredPatt
### Output file must have .conllu extension 
file = "stanza.data.conllu"

f = open(file, "w+")
for x in conll:
        for y in x:
                yy = y[:-1]
                for q in yy:
                        f.write(q + "\t")
                f.write(y[-1] + "\n")
        f.write("\n")
f.close()

### Run is successful - provide name of output file in terminal output
print("\n" + "Output written on " + file + "\n")